In [1]:
import numpy as np
import pandas

ratings = pandas.read_csv('../data/ratings.csv', header=None, names=['User', 'Work', 'Choice'])
works = pandas.read_csv('../data/works.csv', header=None, index_col=0, names=['Title'])

class BaseEmbedding:
    def __init__(self, ratings=None, works=None):
        if ratings is None:
            ratings = pandas.read_csv('../data/ratings.csv', header=None, names=['User', 'Work', 'Choice'])
        if works is None:
            works = pandas.read_csv('../data/works.csv', header=None, index_col=0, names=['Title'])
            
        self.ratings = ratings
        self.works = works
        self.nb_users = ratings['User'].max() + 1
        self.nb_works = ratings['Work'].max() + 1

    def most_similar(self, work_id, topn=8):
        res = self._get_most_similar(work_id, topn)
        similar = self.works.loc[res[1]]
        similar['Similarity'] = pandas.Series(res[0], index=similar.index)
        return similar

# Embedding par SVD de la matrice objets-objets

On va construire une matrice carrée symmétrique comptant le nombre d'utilisateur qui ont donné la même note à deux œuvres, i.e. en position `(i, j)` on aura le nombre d'utilisateur ayant donné la même note à l'œuvre `i` et à l'œuvre `j`. On normalise ensuite cette matrice par la racine carrée du produit des sommes par ligne et par colonne (pour éviter des coefficients disproportionnés sur les œuvres populaires), et on utilise ensuite le résultat d'une SVD comme embedding des œuvres. Enfin, pour déterminer la similarité entre deux œuvres, on utilisera le cosinus des angles entre les vecteurs.

In [2]:
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
import scipy.sparse as sp

class SVDEmbedding(BaseEmbedding):
    def __init__(self, *choices_list, size=100, **kwargs):
        super().__init__(**kwargs)
        self._compute_svd(*choices_list, size=size)
        self._compute_nn()
        
    def _matrix_for(self, choices):
        """Construit une matrice binaire `n_users` x `n_works` telle que en position (i, j)
        se trouve un 0 ssi l'utilisateur `i` à donné à l'œuvre `j` une note contenue dans `choices`"""
        elems = self.ratings[self.ratings['Choice'].isin(choices)].as_matrix()
        return sp.csc_matrix((np.ones(len(elems)), (elems[:,0], elems[:,1])), shape=(self.nb_users, self.nb_works))
    
    def _cooccurrences_for(self, choices):
        """Construit une matrice carrée symmétrique `n_works` x `n_works` telle que en position (i, j)
        se trouve le nombre d'utilisateurs ayant donné aux œuvres `i` et `j` une note contenue dans `choices`."""
        matrix = self._matrix_for(choices)
        return matrix.T.dot(matrix)
    
    def _compute_svd(self, *choices_list, size):
        cooccurrences = np.sum(self._cooccurrences_for(choices) for choices in choices_list)
        # Normalisation
        cooccurrences = np.nan_to_num(cooccurrences / np.sqrt(cooccurrences.sum(axis=1).dot(cooccurrences.sum(axis=0))))
        self._svd = TruncatedSVD(size).fit(cooccurrences)
        
    def _compute_nn(self):
        assert hasattr(self, '_svd')
        self._nn = NearestNeighbors(algorithm='brute', metric='cosine').fit(self._svd.components_.T)
        
    def _get_most_similar(self, work_id, n_neighbors):
        assert hasattr(self, '_svd') and hasattr(self, '_nn')
        res = self._nn.kneighbors(self._svd.components_[:,[work_id]].T, n_neighbors=n_neighbors + 1)
        return (1 - res[0].ravel(), res[1].ravel())

In [3]:
# On considère les ensemble de notes {'dislike'}, {'neutral'}, et {'like', 'favorite'} pour
# compter les cas où la même note a été donnée à des œuvres. N'hésitez pas à expérimenter avec des ensembles différents !
svd_embedding = SVDEmbedding({'dislike'}, {'neutral'}, {'like', 'favorite'}, ratings=ratings, works=works)

/Users/jin/Sites/mangaki/venv/lib/python3.6/site-packages/scipy/sparse/base.py:467: RuntimeWarning: invalid value encountered in true_divide
  return np.true_divide(self.todense(), other)


In [4]:
works[works['Title'].str.contains('Madoka')]

,Title
1344,Puella Magi Madoka Magica
2653,Mahou Shoujo Madoka★Magica
3800,Puella Magi Madoka Magica - The different story
4143,Puella Magi Madoka Magica the Movie Part I: Be...
4438,Mahou Shoujo Madoka★Magica Movie 3: Hangyaku n...
5938,Puella Magi Madoka Magica the Movie Part III: ...
7721,Puella Magi Madoka Magica the Movie Part II: E...


In [8]:
svd_embedding.most_similar(2653)

,Title,Similarity
2653,Mahou Shoujo Madoka★Magica,1.000000
5938,Puella Magi Madoka Magica the Movie Part III: ...,0.897807
5099,Suzumiya Haruhi no Yuuutsu,0.892520
8694,Shinsekai yori,0.889121
1116,Bakemonogatari,0.866234
4143,Puella Magi Madoka Magica the Movie Part I: Be...,0.865144
1239,Clannad: After Story,0.852884
8474,5 centimètres par seconde,0.849994
1312,Steins;Gate,0.846241


# Embedding par `item2vec`

Comme dans https://arxiv.org/abs/1603.04259, on applique simplement le modèle `word2vec` aux utilisateurs. En fait, on considère comme une "phrase" l'ensemble des œuvres auxquelles un utilisateur a donné une certaine note, et on utilise un modèle qui permet d'apprendre un embedding des mots en fonction de leur contexte. L'avantage de cette méthode, c'est qu'on se fiche un peu de la façon dont ces "phrases" sont construites : ainsi on peut utiliser l'ensemble des œuvres auxquelles un utilisateur à donné une certaine note comme ici, mais on pourrait aussi ajouter l'ensemble des œuvres faites par un certain réalisateur, d'un certain genre, etc.

In [10]:
import gensim
import random

class Item2VecEmbedding(BaseEmbedding):
    def __init__(self, *choices_list, size=100, min_count=2, **kwargs):
        super().__init__(**kwargs)
        self.docs = []
        for choices in choices_list:
            elems = self.ratings[self.ratings['Choice'].isin(choices)][['User', 'Work']].as_matrix()
            docs = [[] for _ in range(self.nb_users)]
            for user_id, work_id in elems:
                docs[user_id].append(str(work_id))
            self.docs.extend(docs)
        max_len = max(len(doc) for doc in self.docs)
        self._word2vec = gensim.models.Word2Vec(self.docs, size=size, window=max_len+1, min_count=min_count, iter=10, sg=1)
        
    def _get_most_similar(self, work_id, n_neighbors):
        res = self._word2vec.most_similar(str(work_id), topn=n_neighbors)
        return ([1.] + [x for _, x in res], [work_id] + [int(x) for x, _ in res])

In [11]:
# On considère les ensemble {'neutral'}, {'dislike'} et {'like', 'favorite'} pour créer les "phrases".
# N'hésitez pas à expérimenter avec des valeurs différentes
embedding = Item2VecEmbedding({'neutral'}, {'dislike'}, {'like', 'favorite'}, ratings=ratings, works=works)

In [27]:
embedding.most_similar(123)

,Title,Similarity
123,GTO - Great Teacher Onizuka,1.000000
3509,FullMetal Alchemist,0.867304
8363,Dragon Ball,0.852581
1668,Bakuman,0.849891
8034,One piece - Edition originale,0.849425
3819,Soul Eater,0.843967
424,Attaque Des Titans (l'),0.843405
7169,Death note,0.839352
3717,Eye Shield 21,0.814974
